https://towardsdatascience.com/train-a-gpt-2-transformer-to-write-harry-potter-books-edf8b2e3f3db

Preprocessing the text

In [1]:
import os
import re

In [5]:
%%bash
wget https://raw.githubusercontent.com/priya-dwivedi/Deep-Learning/master/GPT2-HarryPotter-Training/data_cleanup/merged_books.txt

--2020-10-13 05:28:35--  https://raw.githubusercontent.com/priya-dwivedi/Deep-Learning/master/GPT2-HarryPotter-Training/data_cleanup/merged_books.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2972019 (2.8M) [text/plain]
Saving to: ‘merged_books.txt’

     0K .......... .......... .......... .......... ..........  1% 1.35M 2s
    50K .......... .......... .......... .......... ..........  3% 2.87M 1s
   100K .......... .......... .......... .......... ..........  5% 8.65M 1s
   150K .......... .......... .......... .......... ..........  6% 11.0M 1s
   200K .......... .......... .......... .......... ..........  8% 4.71M 1s
   250K .......... .......... .......... .......... .......... 10% 13.9M 1s
   300K .......... .......... .......... .......... ........

In [6]:
with open("merged_books.txt",'r') as f:
    string = f.read()
    print(string)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [7]:
def clean_text(string):
    pattern = '(page|PAGE|Page)(\s+\|\s+)([0-9]+)(.*)$'
    output_cleaned = re.sub('\s$', '', string, flags=re.MULTILINE)
    p=re.compile(pattern,re.MULTILINE)
    output_cleaned = p.sub(" ",output_cleaned)
    return output_cleaned

In [8]:
check = clean_text(string)

In [9]:
check

Output hidden; open in https://colab.research.google.com to view.

In [10]:
with open("output_cleaned.txt",'w') as f:
     f.writelines(check)

In [11]:
train_doc =[]
val_doc =[]

In [12]:
with open("output_cleaned.txt",'r') as f:
    file_input=f.readlines()

In [13]:
file_input

['\n',
 'THE BOY WHO LIVED\n',
 'Mr. and Mrs. Dursley, of number four, Privet Drive,\n',
 'were proud to say that they were perfectly normal,\n',
 'thank you very much. They were the last people you’d\n',
 'expect to be involved in anything strange or\n',
 'mysterious, because they just didn’t hold with such\n',
 'nonsense.\n',
 'Mr. Dursley was the director of a firm called\n',
 'Grunnings, which made drills. He was a big, beefy\n',
 'man with hardly any neck, although he did have a\n',
 'very large mustache. Mrs. Dursley was thin and\n',
 'blonde and had nearly twice the usual amount of\n',
 'neck, which came in very useful as she spent so\n',
 'much of her time craning over garden fences, spying\n',
 'on the neighbors. The Dursley s had a small son\n',
 'called Dudley and in their opinion there was no finer\n',
 'boy anywhere.\n',
 'The Dursleys had everything they wanted, but they\n',
 'also had a secret, and their greatest fear was that\n',
 'somebody would discover it. They didn’

In [14]:
count = 0
for cnt, line in enumerate(file_input):
#         print(cnt)
        if cnt <= len(file_input)*0.90:
            train_doc.append(line)
        else:
            val_doc.append(line)

In [15]:
len(train_doc)

54937

In [16]:
## Write to file
f = open('train_harry.txt', "w+")
count = 0
for line in train_doc:
    count=count+1
    f.write(str(line))
    f.write("\n")
    
f.close()
print(count)

54937


In [17]:
## Write to file
f = open('val_harry.txt', "w+")
count = 0
for line in val_doc:
    count=count+1
    f.write(str(line))
    f.write("\n")
    
f.close()
print(count)

6103
